In [2]:
import pickle
import torchvision.transforms as transforms

# Function to unpack the cifar-10 dataset.
def unpickle(file):
    with open(file, 'rb') as file:
        data = pickle.load(file, encoding='bytes')
    return data[b'data'], data[b'labels']

In [2]:
import os
from torch.utils.data import Dataset
#from utils import unpickle


class LoadTrainingData(Dataset):
    def __init__(self):
        self.trainX = []
        self.trainY = []

        data_dir = './cifar-10/training batches'
        batches = os.listdir(data_dir)

        for batch in batches:
            batch_data, batch_labels = unpickle(os.path.join(data_dir, batch))
            self.trainX.extend(batch_data)
            self.trainY.extend(batch_labels)

    def __getitem__(self, item):
        return self.trainX[item], self.trainY[item]

    def __len__(self):
        return len(self.trainX)

In [3]:
data_dir = './cifar-10/'

transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Normalize the test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=opt.batch_size_train, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=opt.batch_size_test, shuffle=False, num_workers=2)

NameError: name 'torchvision' is not defined

In [3]:
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict


class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.layers = nn.Sequential(OrderedDict([
            ('layer1', nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.BatchNorm2d(num_features=32),
                nn.MaxPool2d(kernel_size=2, stride=2)
            )),
            ('layer2', nn.Sequential(
                nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.BatchNorm2d(num_features=64),
                nn.MaxPool2d(kernel_size=2, stride=2)
            )),
            ('fc1', nn.Sequential(
                nn.Linear(in_features=8 * 8 * 64, out_features=1024),
                nn.ReLU()
            ))
        ]))
        self.out = nn.Linear(in_features=1024, out_features=10)

    def forward(self, x):
        for name, module in self.layers.named_children():
            x = module(x)

            if name == 'layer2':
                x = x.view(x.size(0), -1)

        x = self.out(x)
        return F.softmax(x, dim=1)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

#from load_data import LoadTrainingData
#from model import CNNModel

def train_model(model, data, epoch, batch_size):
    # define the loss function and back propagation algorithm
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.2, momentum=0.9)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.01)

    for e in range(epoch):
        for param_group in optimizer.param_groups:
            lr = param_group['lr']
        print('[EPOCH: %d, Learning Rate: %f]' % (e+1, lr))
        print()
        for i, dataset in enumerate(data):
            inputs, lbl = dataset
            inputs, lbl = inputs.view(batch_size, 3, 32, 32).to('cuda', dtype=torch.float), lbl.view(-1).to('cuda')

            if torch.cuda.is_available():
                inputs, lbl = inputs.cuda(), lbl.cuda()

            # set the gradient for each parameters zero
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, lbl)
            loss.backward()
            optimizer.step()
            print('-[step: %d, loss: %f]' % (i+1, loss.item()))
        scheduler.step()


    print ('Finished Training')


if __name__ == '__main__':
    cnn = CNNModel()
    batch = 2000
    if torch.cuda.is_available():
        cnn.cuda()

    trainingDataset = LoadTrainingData()
    dataLoader = DataLoader(
        dataset=trainingDataset,
        batch_size=batch,
        shuffle=True,
        num_workers=2
    )

    train_model(cnn, dataLoader, epoch=40, batch_size=batch)

    # save model
    torch.save(cnn.state_dict(), './trained_model.pth')

[EPOCH: 1, Learning Rate: 0.200000]

-[step: 1, loss: 2.301567]
-[step: 2, loss: 2.292770]
-[step: 3, loss: 2.276508]
-[step: 4, loss: 2.257814]
-[step: 5, loss: 2.233335]
-[step: 6, loss: 2.209933]
-[step: 7, loss: 2.192120]
-[step: 8, loss: 2.152890]
-[step: 9, loss: 2.165091]
-[step: 10, loss: 2.145174]
-[step: 11, loss: 2.115953]
-[step: 12, loss: 2.115658]
-[step: 13, loss: 2.090153]
-[step: 14, loss: 2.115158]
-[step: 15, loss: 2.091876]
-[step: 16, loss: 2.054836]
-[step: 17, loss: 2.062256]
-[step: 18, loss: 2.061877]
-[step: 19, loss: 2.056669]
-[step: 20, loss: 2.013210]
-[step: 21, loss: 2.025482]
-[step: 22, loss: 2.030421]
-[step: 23, loss: 2.026195]
-[step: 24, loss: 1.997824]
-[step: 25, loss: 1.997454]
[EPOCH: 2, Learning Rate: 0.200000]

-[step: 1, loss: 1.981152]
-[step: 2, loss: 1.975069]
-[step: 3, loss: 1.974908]
-[step: 4, loss: 1.985329]
-[step: 5, loss: 1.981535]
-[step: 6, loss: 1.974297]
-[step: 7, loss: 1.944162]
-[step: 8, loss: 1.962361]
-[step: 9, loss: 1.

-[step: 7, loss: 1.608867]
-[step: 8, loss: 1.586722]
-[step: 9, loss: 1.601051]
-[step: 10, loss: 1.607148]
-[step: 11, loss: 1.596421]
-[step: 12, loss: 1.599518]
-[step: 13, loss: 1.601556]
-[step: 14, loss: 1.596440]
-[step: 15, loss: 1.596069]
-[step: 16, loss: 1.599000]
-[step: 17, loss: 1.611371]
-[step: 18, loss: 1.601018]
-[step: 19, loss: 1.603964]
-[step: 20, loss: 1.608648]
-[step: 21, loss: 1.604140]
-[step: 22, loss: 1.606848]
-[step: 23, loss: 1.597360]
-[step: 24, loss: 1.603546]
-[step: 25, loss: 1.602724]
[EPOCH: 13, Learning Rate: 0.200000]

-[step: 1, loss: 1.592598]
-[step: 2, loss: 1.584652]
-[step: 3, loss: 1.586247]
-[step: 4, loss: 1.586319]
-[step: 5, loss: 1.571583]
-[step: 6, loss: 1.582771]
-[step: 7, loss: 1.576900]
-[step: 8, loss: 1.582707]
-[step: 9, loss: 1.583273]
-[step: 10, loss: 1.580014]
-[step: 11, loss: 1.576215]
-[step: 12, loss: 1.578402]
-[step: 13, loss: 1.574401]
-[step: 14, loss: 1.589114]
-[step: 15, loss: 1.585638]
-[step: 16, loss: 1.59

-[step: 15, loss: 1.525694]
-[step: 16, loss: 1.526849]
-[step: 17, loss: 1.525555]
-[step: 18, loss: 1.521559]
-[step: 19, loss: 1.527334]
-[step: 20, loss: 1.526304]
-[step: 21, loss: 1.515346]
-[step: 22, loss: 1.525240]
-[step: 23, loss: 1.518541]
-[step: 24, loss: 1.521445]
-[step: 25, loss: 1.535055]
[EPOCH: 24, Learning Rate: 0.002000]

-[step: 1, loss: 1.539808]
-[step: 2, loss: 1.526635]
-[step: 3, loss: 1.526001]
-[step: 4, loss: 1.525193]
-[step: 5, loss: 1.524613]
-[step: 6, loss: 1.528049]
-[step: 7, loss: 1.531738]
-[step: 8, loss: 1.524133]
-[step: 9, loss: 1.522188]
-[step: 10, loss: 1.539484]
-[step: 11, loss: 1.525808]
-[step: 12, loss: 1.529989]
-[step: 13, loss: 1.524159]
-[step: 14, loss: 1.531269]
-[step: 15, loss: 1.524484]
-[step: 16, loss: 1.518479]
-[step: 17, loss: 1.525067]
-[step: 18, loss: 1.517382]
-[step: 19, loss: 1.525409]
-[step: 20, loss: 1.526881]
-[step: 21, loss: 1.524439]
-[step: 22, loss: 1.528955]
-[step: 23, loss: 1.519753]
-[step: 24, loss: 1

-[step: 22, loss: 1.531090]
-[step: 23, loss: 1.531733]
-[step: 24, loss: 1.529702]
-[step: 25, loss: 1.522178]
[EPOCH: 35, Learning Rate: 0.002000]

-[step: 1, loss: 1.524109]
-[step: 2, loss: 1.521720]
-[step: 3, loss: 1.529190]
-[step: 4, loss: 1.525542]
-[step: 5, loss: 1.541047]
-[step: 6, loss: 1.530302]
-[step: 7, loss: 1.514991]
-[step: 8, loss: 1.519609]
-[step: 9, loss: 1.529602]
-[step: 10, loss: 1.525289]
-[step: 11, loss: 1.525458]
-[step: 12, loss: 1.528204]
-[step: 13, loss: 1.522348]
-[step: 14, loss: 1.514276]
-[step: 15, loss: 1.522035]
-[step: 16, loss: 1.525554]
-[step: 17, loss: 1.527093]
-[step: 18, loss: 1.524886]
-[step: 19, loss: 1.522889]
-[step: 20, loss: 1.533844]
-[step: 21, loss: 1.525225]
-[step: 22, loss: 1.519916]
-[step: 23, loss: 1.527823]
-[step: 24, loss: 1.524634]
-[step: 25, loss: 1.530548]
[EPOCH: 36, Learning Rate: 0.002000]

-[step: 1, loss: 1.518864]
-[step: 2, loss: 1.524131]
-[step: 3, loss: 1.526880]
-[step: 4, loss: 1.531890]
-[step: 5, lo

In [6]:
import torch
import numpy as np

# from utils import unpickle
# from model import CNNModel

def main():
    trained_model = './trained_model.pth'
    test_batch_dir = './cifar-10/test_batch'
    
    classifier = CNNModel()
    classifier.load_state_dict(torch.load(trained_model))
    classifier.cuda()
    classifier.eval()

    test_x, test_y = unpickle(test_batch_dir)
    test_x, test_y = torch.tensor(np.reshape(test_x, (len(test_x), 3, 32, 32))).to('cuda', dtype=torch.float), torch.tensor(test_y).cuda()

    classes = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

    # calculating the accuracy of our classifier;
    print("Calculating accuracy...")
    correct = 0
    total = len(test_x)

    with torch.no_grad():
        out = classifier(test_x)
        _, predicted = torch.max(out, 1)

        # calculate the total accuracy
        correct += (predicted == test_y).sum().item()
        print('Accuracy: %5d %%' % (correct / total * 100))


if __name__ == '__main__':
    main()

Calculating accuracy...
Accuracy:    76 %
